In [1]:
%load_ext autoreload
%autoreload 2

In [107]:
import jax
import jax.numpy as np
import jraph
import flax.linen as nn

from functools import partial

In [108]:
import sys
sys.path.append("../")

from models.graph_utils import nearest_neighbors
import jaxkdtree

In [109]:
n_nodes = 5000

x = np.load("data/halos_small.npy")[:4, :n_nodes, :3]

x.shape

(4, 5000, 3)

In [110]:
%%timeit
# kD-tree
res = jax.vmap(jaxkdtree.kNN, in_axes=(0,None,None))(x[:4, :, :3], 16, 100.0)
res.shape

10.9 ms ± 74.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%%timeit
# Pairwise distances
sources, targets, dist = jax.vmap(nearest_neighbors, in_axes=(0, None))(x[:4, :, :3], 16)
np.array([sources, targets])

23.1 ms ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [112]:
sources, targets, dist = jax.vmap(nearest_neighbors, in_axes=(0, None))(x[:4, :, :3], 8)

In [113]:
@jax.jit
def nearest_neighbors_kd(x, k, max_radius=2000.):
    res = jaxkdtree.kNN(x, k, max_radius)
    sources = np.repeat(np.arange(x.shape[0]), k)
    targets = res.reshape(-1)
    return sources, targets

In [114]:
sources_knn, targets_knn = jax.vmap(nearest_neighbors_kd, in_axes=(0,None,None))(x[:4, :, :3], 8, 1000.)

In [115]:
np.allclose(sources, sources_knn)

Array(True, dtype=bool)